In [1]:
library(tidyverse)
library(hash)
library(gprofiler2)
library(viridis)
library(VennDiagram)
library(XML)
library(httr)
library(stringi)

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.7
v tidyr   1.1.4     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'hash' was built under R version 3.6.3"hash-3.0.1 provided by Decision Patterns

Loading required package: viridisLite

Attaching package: 'viridis'

The following object is masked from 'package:viridisLite':

    viridis.map

Loading required package: grid
Loading required package: futile.logger


In [2]:
alpha <- 1e-5
results_prefix <- '/data/projects/julia.pratt/CS1_genomeSelection/scripts/revigo/PCs/COMPLETE/'
sourcepreferences <- NULL

In [3]:
tissues <- c('Leaf')
years <- c('2018')
phenos <- c('Anthesis', 'Veraison', 'Harvest')
rootstocks <- c('Ungrafted', '1103P', '3309C', 'SO4')
dir <- c('Up', 'Down')
prefix <- '/data/projects/julia.pratt/CS1_genomeSelection/scripts/PCs_for_julia/'

stats <- list()
dats <- list()
skipped <- list()

In [4]:
# Convert vitis (v3) to vit_ (v1s)
aliasmap <- data.frame(read_tsv('v3_mapNamesToAlias.tsv', col_names = FALSE))
colnames(aliasmap) <- c('12Xv1', 'VCOST.v3')
aliashash <- hash(keys=aliasmap$'12Xv1', values=aliasmap$'VCOST.v3')

convert <- function(i){
  vitvi <- as.character(i)
  vitvi.alias <- unlist(strsplit(aliashash[[vitvi]], ','))
  vitvi.alias.reduced <- unlist(vitvi.alias[grepl('VIT_', vitvi.alias, fixed=TRUE)])
  if(length(vitvi.alias.reduced) == 0){
    #print(paste("no vit_ name located for", vitvi))
  }
  return(vitvi.alias.reduced)
}

Parsed with column specification:
cols(
  X1 = col_character(),
  X2 = col_character()
)
Warning message:
"15 parsing failures.
 row col  expected    actual                     file
9454  -- 2 columns 1 columns 'v3_mapNamesToAlias.tsv'
9459  -- 2 columns 1 columns 'v3_mapNamesToAlias.tsv'
9465  -- 2 columns 1 columns 'v3_mapNamesToAlias.tsv'
9466  -- 2 columns 1 columns 'v3_mapNamesToAlias.tsv'
9468  -- 2 columns 1 columns 'v3_mapNamesToAlias.tsv'
.... ... ......... ......... ........................
See problems(...) for more details.
"

In [5]:
# Use gprofiler2 to perform gene ontology term enrichment
go <- function(prepped.data, alpha=0.05, sourcespreferences=NULL) {
    # upregulated genes first
    d <- gost(query = prepped.data, 
                    organism = "vvinifera", ordered_query = FALSE, 
                    multi_query = FALSE, significant = TRUE, exclude_iea = FALSE, 
                    measure_underrepresentation = FALSE, evcodes = FALSE, 
                    user_threshold = alpha, correction_method = "fdr", 
                    domain_scope = "annotated", custom_bg = NULL, 
                    numeric_ns = "", sources = sourcepreferences, as_short_link = FALSE)
    return(d)
}

In [6]:
# cycle through all the data
# this is essentially the gene ontology pipeline

for (f in list.files(prefix) ) {
    # read in the data
    fullpath <- paste(prefix, f, sep='')
    data <- read.csv(fullpath)
    data.clean <- unlist(lapply(X=data$x, FUN=convert))
    fname <- substr(f,1,nchar(f)-4)
    
    # run gene ontology
    if(length(data.clean) > 0){
        res <- go(data.clean, alpha)
        if(length(res$result$source) > 0){
            # save info
            stats[[fname]] <- res$result
            dats[[fname]] <- data.clean
            print(fname)
        }
        # keep track of which years & time points are skipped to prevent future errors
        else{skipped <- append(skipped, fname)}
    }
    else{skipped <- append(skipped, fname)}
}

[1] "PC105_high_loaders"


No results to show
Please make sure that the organism is correct or set significant = FALSE


[1] "PC10_high_loaders"
[1] "PC10_low_loaders"
[1] "PC11_high_loaders"
[1] "PC11_low_loaders"
[1] "PC12_high_loaders"
[1] "PC12_low_loaders"


No results to show
Please make sure that the organism is correct or set significant = FALSE


[1] "PC13_low_loaders"


...

[1] "PC6_high_loaders"
[1] "PC6_low_loaders"


No results to show
Please make sure that the organism is correct or set significant = FALSE


[1] "PC7_low_loaders"
[1] "PC8_high_loaders"
[1] "PC8_low_loaders"


No results to show
Please make sure that the organism is correct or set significant = FALSE


[1] "PC97_low_loaders"
[1] "PC9_high_loaders"
[1] "PC9_low_loaders"


In [7]:
df <- setNames(data.frame(matrix(ncol = 3, nrow = 0)), c("name", "numTerms", "numGenes"))
for (name in names(stats)){
    df[nrow(df) + 1,] = c(name, dim(stats[[name]])[1], length(dats[[name]]))
    print(stats[[name]][['term_id']])
    print(stats[[name]][['term_name']])
    print("=======================")
}

[1] "GO:0009451"
[1] "RNA modification"
[1] "======================="
 [1] "GO:0009408" "GO:0051259" "GO:0009266" "GO:0042542" "GO:0006457"
 [6] "GO:0000302" "GO:0006970" "GO:0009651" "GO:0009628" "GO:1901700"
[11] "GO:0010035" "GO:0043933" "GO:0065003" "GO:0006979" "GO:0043621"
[16] "GO:0051082" "KEGG:04141"
 [1] "response to heat"                           
 [2] "protein complex oligomerization"            
 [3] "response to temperature stimulus"           
 [4] "response to hydrogen peroxide"              
 [5] "protein folding"                            
 [6] "response to reactive oxygen species"        
 [7] "response to osmotic stress"                 
 [8] "response to salt stress"                    
 [9] "response to abiotic stimulus"               
[10] "response to oxygen-containing compound"     
[11] "response to inorganic substance"            
[12] "protein-containing complex organization"    
[13] "protein-containing complex assembly"        
[14] "response to oxidativ

In [8]:
df2 <- NULL
for (name in names(stats)){
    df2 <- rbind(df2, data.frame(
        "name" = rep(name, dim(stats[[name]])[1]),
        "GO.source" = stats[[name]] %>% select('source'),
        "GO.term.id" = stats[[name]] %>% select('term_id'),
        "GO.term.aname" = stats[[name]] %>% select('term_name'),
        "p.value" = stats[[name]] %>% select('p_value'))
    )
    
    tmp <- stats[[name]] %>% select(term_id, p_value)
    finalname <- paste(results_prefix, paste("REVIGO", name, sep="_"), ".csv", sep="")
    write.table(tmp, "tmp.txt", quote=FALSE, row.names = FALSE, col.names = FALSE)
    filedata <-readChar("tmp.txt",file.info("tmp.txt")$size)
    
    httr::POST(
      url = "http://revigo.irb.hr/Revigo.aspx",
      body = list(
        cutoff = "0.5",
        valueType = "pvalue",
        speciesTaxon = "0",
        measure = "SIMREL",
        goList = filedata
      ),
      # application/x-www-form-urlencoded
      encode = "form"
    ) -> res

    dat <- httr::content(res, encoding = "UTF-8")

    # Write results to a file
    dat <- stri_replace_all_fixed(dat, "\r", "")
    cat(dat, file='tmpresults.html', fill = FALSE)
    resultsrevigo <- readHTMLTable('tmpresults.html')
    
    fullresultsrevigo <- setNames(data.frame(matrix(ncol = 4, nrow = 0)), c("Term.ID", "Name", "Value", "Category"))
    if(!is.null(resultsrevigo$BiologicalProcess)){
        resultsrevigo$BiologicalProcess <- resultsrevigo$BiologicalProcess %>% 
                                            filter(Eliminated=='False') %>% 
                                            select(`Term ID`, Name, Value)
        resultsrevigo$BiologicalProcess['Category'] <- rep('BiologicalProcess', dim(resultsrevigo$BiologicalProcess)[1])
        fullresultsrevigo <- rbind(fullresultsrevigo, resultsrevigo$BiologicalProcess)
    }
    if(!is.null(resultsrevigo$CellularComponent)){
        resultsrevigo$CellularComponent <- resultsrevigo$CellularComponent %>% 
                                            filter(Eliminated=='False') %>% 
                                            select(`Term ID`, Name, Value)
        resultsrevigo$CellularComponent['Category'] <- rep('CellularComponent', dim(resultsrevigo$CellularComponent)[1])
        fullresultsrevigo <- rbind(fullresultsrevigo, resultsrevigo$CellularComponent)
    }
    if(!is.null(resultsrevigo$MolecularFunction)){
        resultsrevigo$MolecularFunction <- resultsrevigo$MolecularFunction %>% 
                                            filter(Eliminated=='False') %>% 
                                            select(`Term ID`, Name, Value)
        resultsrevigo$MolecularFunction['Category'] <- rep('MolecularFunction', dim(resultsrevigo$MolecularFunction)[1])
        fullresultsrevigo <- rbind(fullresultsrevigo, resultsrevigo$MolecularFunction)
    }
    
    write.csv(data.frame(fullresultsrevigo), finalname, row.names=FALSE)
}

Warning message in stri_replace_all_fixed(dat, "\r", ""):
"argument is not an atomic vector; coercing"Warning message in stri_replace_all_fixed(dat, "\r", ""):
"argument is not an atomic vector; coercing"Warning message in stri_replace_all_fixed(dat, "\r", ""):
"argument is not an atomic vector; coercing"Warning message in stri_replace_all_fixed(dat, "\r", ""):
"argument is not an atomic vector; coercing"Warning message in stri_replace_all_fixed(dat, "\r", ""):
"argument is not an atomic vector; coercing"Warning message in stri_replace_all_fixed(dat, "\r", ""):
"argument is not an atomic vector; coercing"Warning message in stri_replace_all_fixed(dat, "\r", ""):
"argument is not an atomic vector; coercing"Warning message in stri_replace_all_fixed(dat, "\r", ""):
"argument is not an atomic vector; coercing"Warning message in stri_replace_all_fixed(dat, "\r", ""):
"argument is not an atomic vector; coercing"Warning message in stri_replace_all_fixed(dat, "\r", ""):
"argument is not an atom

In [9]:
write.csv(df, paste(results_prefix, "GO_pcs_stats.csv", sep=""))
write.csv(df2, paste(results_prefix, "GO_pcs_go_term_list.csv", sep=""))